In [ ]:
# default_exp transforms.sampling

# Sampling
> Data sampling methods.

In [ ]:
#hide
from nbdev.showdoc import *

## Negative Sampling

In [ ]:
#export
from abc import *
from pathlib import Path
import pickle
import os
from tqdm import trange
from collections import Counter
import numpy as np
import pandas as pd

In [ ]:
#export
def simple_negative_sampling(data,
                             num_negatives=4,
                             binarization=False,
                             feedback_column='RATING'):
  
  # Get a list of all Item IDs
  all_itemsIds = data['ITEMID'].unique()

  # Placeholders that will hold the data
  users, items, labels = [], [], []

  if binarization:
    data.loc[:,feedback_column] = 1

  user_item_set = set(zip(data['USERID'], data['ITEMID'], data[feedback_column]))

  for (u, i, r) in user_item_set:
    users.append(u)
    items.append(i)
    labels.append(r)
    for _ in range(num_negatives):
      # randomly select an item
      negative_item = np.random.choice(all_itemsIds) 
      # check that the user has not interacted with this item
      while (u, negative_item) in user_item_set:
          negative_item = np.random.choice(all_itemsIds)
      users.append(u)
      items.append(negative_item)
      labels.append(0) # items not interacted with are negative
  ns_data = pd.DataFrame(list(zip(users, items, labels)),
                         columns=['USERID','ITEMID',feedback_column])
  return ns_data

In [ ]:
#export
class AbstractNegativeSampler(metaclass=ABCMeta):
    def __init__(self, train, val, test, user_count, item_count, sample_size, seed, flag, save_folder):
        self.train = train
        self.val = val
        self.test = test
        self.user_count = user_count
        self.item_count = item_count
        self.sample_size = sample_size
        self.seed = seed
        self.flag = flag
        self.save_path = save_path

    @classmethod
    @abstractmethod
    def code(cls):
        pass

    @abstractmethod
    def generate_negative_samples(self):
        pass

    def get_negative_samples(self):
        savefile_path = self._get_save_path()
        print("Negative samples don't exist. Generating.")
        seen_samples, negative_samples = self.generate_negative_samples()
        with savefile_path.open('wb') as f:
            pickle.dump([seen_samples, negative_samples], f)
        return seen_samples, negative_samples

    def _get_save_path(self):
        folder = Path(self.save_path)
        if not folder.is_dir():
            folder.mkdir(parents=True)
        # filename = '{}-sample_size{}-seed{}-{}.pkl'.format(
        #     self.code(), self.sample_size, self.seed, self.flag)
        filename = 'negative_samples_{}.pkl'.format(self.flag)
        return folder.joinpath(filename)

In [ ]:
#export
class RandomNegativeSampler(AbstractNegativeSampler):
    @classmethod
    def code(cls):
        return 'random'

    def generate_negative_samples(self):
        assert self.seed is not None, 'Specify seed for random sampling'
        np.random.seed(self.seed)
        num_samples = 2 * self.user_count * self.sample_size
        all_samples = np.random.choice(self.item_count, num_samples) + 1

        seen_samples = {}
        negative_samples = {}
        print('Sampling negative items randomly...')
        j = 0
        for i in trange(self.user_count):
            user = i + 1
            seen = set(self.train[user])
            seen.update(self.val[user])
            seen.update(self.test[user])
            seen_samples[user] = seen

            samples = []
            while len(samples) < self.sample_size:
                item = all_samples[j % num_samples]
                j += 1
                if item in seen or item in samples:
                    continue
                samples.append(item)
            negative_samples[user] = samples

        return seen_samples, negative_samples

In [ ]:
#export
class PopularNegativeSampler(AbstractNegativeSampler):
    @classmethod
    def code(cls):
        return 'popular'

    def generate_negative_samples(self):
        assert self.seed is not None, 'Specify seed for random sampling'
        np.random.seed(self.seed)
        popularity = self.items_by_popularity()
        items = list(popularity.keys())
        total = 0
        for i in range(len(items)):
            total += popularity[items[i]]
        for i in range(len(items)):
            popularity[items[i]] /= total
        probs = list(popularity.values())
        num_samples = 2 * self.user_count * self.sample_size
        all_samples = np.random.choice(items, num_samples, p=probs)

        seen_samples = {}
        negative_samples = {}
        print('Sampling negative items by popularity...')
        j = 0
        for i in trange(self.user_count):
            user = i + 1
            seen = set(self.train[user])
            seen.update(self.val[user])
            seen.update(self.test[user])
            seen_samples[user] = seen

            samples = []
            while len(samples) < self.sample_size:
                item = all_samples[j % num_samples]
                j += 1
                if item in seen or item in samples:
                    continue
                samples.append(item)
            negative_samples[user] = samples

        return seen_samples, negative_samples

    def items_by_popularity(self):
        popularity = Counter()
        self.users = sorted(self.train.keys())
        for user in self.users:
            popularity.update(self.train[user])
            popularity.update(self.val[user])
            popularity.update(self.test[user])

        popularity = dict(popularity)
        popularity = {k: v for k, v in sorted(popularity.items(), key=lambda item: item[1], reverse=True)}
        return popularity

## Graph Sampling

In [ ]:
#export
import random
import numpy as np
from collections import deque

In [ ]:
#export
class ForestFireSampler:
    """An implementation of forest fire sampling. The procedure is a stochastic
    snowball sampling method where the expansion is proportional to the burning probability. 
    `"For details about the algorithm see this paper." <https://cs.stanford.edu/people/jure/pubs/sampling-kdd06.pdf>`_
    Inspiration credit: 
        littleballoffur
        https://github.com/benedekrozemberczki/littleballoffur
    Args:
        number_of_nodes (int): Number of sampled nodes. Default is 100.
        p (float): Burning probability. Default is 0.4.
        seed (int): Random seed. Default is 42.
    """
    def __init__(self, number_of_nodes: int=100, p: float=0.4, seed: int=42, max_visited_nodes_backlog: int=100,
                 restart_hop_size: int = 10):
        self.number_of_nodes = number_of_nodes
        self.p = p
        self.seed = seed
        self._set_seed() 
        self.restart_hop_size = restart_hop_size
        self.max_visited_nodes_backlog = max_visited_nodes_backlog

    def _set_seed(self):
        random.seed(self.seed)
        np.random.seed(self.seed)

    def _create_node_sets(self, graph):
        """
        Create a starting set of nodes.
        """
        self._sampled_nodes = set()
        self._set_of_nodes = set(range(graph.number_of_nodes()))
        self._visited_nodes = deque(maxlen=self.max_visited_nodes_backlog)

    def get_neighbors(self, graph, node):
        return list(graph.neighbors(node))

    def _start_a_fire(self, graph):
        """
        Starting a forest fire from a single node.
        """
        remaining_nodes = list(self._set_of_nodes.difference(self._sampled_nodes))
        seed_node = random.choice(remaining_nodes)
        self._sampled_nodes.add(seed_node)
        node_queue = deque([seed_node])
        while len(self._sampled_nodes) < self.number_of_nodes:
            if len(node_queue) == 0:
                node_queue = deque([self._visited_nodes.popleft()
                              for k in range(min(self.restart_hop_size, len(self._visited_nodes)))])
                if len(node_queue) == 0:
                    # print('Warning: could not collect the required number of nodes. The fire could not find enough nodes to burn.')
                    break
            top_node = node_queue.popleft()
            self._sampled_nodes.add(top_node)
            neighbors = set(self.get_neighbors(graph, top_node))
            unvisited_neighbors = neighbors.difference(self._sampled_nodes)
            score = np.random.geometric(self.p)
            count = min(len(unvisited_neighbors), score)
            burned_neighbors = random.sample(unvisited_neighbors, count)
            self._visited_nodes.extendleft(unvisited_neighbors.difference(set(burned_neighbors)))
            for neighbor in burned_neighbors:
                if len(self._sampled_nodes) >= self.number_of_nodes:
                    break
                node_queue.extend([neighbor])

In [ ]:
#export
class RandomWalkWithRestartSampler:
    """An implementation of node sampling by random walks with restart. The 
    process is a discrete random walker on nodes which teleports back to the
    staring node with a fixed probability. This results in a connected subsample
    from the original input graph. `"For details about the algorithm see this 
    paper." <https://cs.stanford.edu/people/jure/pubs/sampling-kdd06.pdf>`_
    Inspiration credit: 
        littleballoffur
        https://github.com/benedekrozemberczki/littleballoffur
    Args:
        number_of_nodes (int): Number of nodes. Default is 100.
        seed (int): Random seed. Default is 42.
        p (float): Restart probability. Default is 0.1.
    """
    def __init__(self, number_of_nodes: int=100, seed: int=42, p: float=0.1):
        self.number_of_nodes = number_of_nodes
        self.seed = seed
        self.p = p
        self._set_seed()

    def _set_seed(self):
        random.seed(self.seed)

    def get_neighbors(self, graph, node):
        return list(graph.neighbors(node))

    def get_random_neighbor(self, graph, node):
        return random.choice(self.get_neighbors(graph, node))

    def get_nodes(self, graph):
        return list(graph.nodes)

    def get_number_of_nodes(self, graph):
        return graph.number_of_nodes()

    def _create_initial_node_set(self, graph, start_node):
        """
        Choosing an initial node.
        """
        self._set_of_nodes = set(self.get_nodes(graph))

        if start_node is not None:
            if start_node >= 0 and start_node < self.get_number_of_nodes(graph):
                self._current_node = start_node
                self._sampled_nodes = set([self._current_node])
            else:
                raise ValueError("Starting node index is out of range.")
        else:
            self._current_node = random.choice(range(self.get_number_of_nodes(graph)))
            self._sampled_nodes = set([self._current_node])
        self._initial_node = self._current_node

    def _do_a_step(self, graph):
        """
        Doing a single random walk step.
        """
        score = random.uniform(0, 1)
        if score < self.p:
            self._current_node = self._initial_node
        else:
            new_node = self.get_random_neighbor(graph, self._current_node)
            self._sampled_nodes.add(new_node)
            self._current_node = new_node

In [ ]:
#hide
!pip install -q watermark
%reload_ext watermark
%watermark -a "Sparsh A." -m -iv -u -t -d

Author: Sparsh A.

Last updated: 2021-12-18 09:51:57

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 5.4.104+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit

pandas : 1.1.5
IPython: 5.5.0
numpy  : 1.19.5

